In [ ]:
library(ggplot2)
library(reshape2)
library(dplyr)
library(nlme)
library(tidyr)
library(latex2exp)

theme_set(theme_minimal(base_size = 13))
phi = 2 / (1 + sqrt(5))

In [ ]:
path = 'input/_output_pipe_freq/merged.agg.csv.bz2'
#path = 'input/_output_pipe_freq_1M/merged.agg.csv.bz2'
df = read.csv(bzfile(path), stringsAsFactors=FALSE)
df$log_frequency = log(df$frequency)
df$n_subjects = gsub('count_in_', '', df$column)
df$n_subjects = as.integer(gsub('count', '666', df$n_subjects))
df$column = NULL

_Make choice here!_

In [ ]:
# Which VAE?
df = df[df$model == 'basic',]
#df = df[df$model == 'count_match',]

Here we add "pseudocounts" of half the minimum nonzero value so that R doesn't choke on Pgen/Ppost calculation when they are zero.

In [ ]:
df[is.na(df$Pgen),'Pgen'] = min(df$Pgen, na.rm = TRUE)/2
df[is.na(df$Ppost),'Ppost'] = min(df$Ppost, na.rm = TRUE)/2

In [ ]:
df$Pvae = exp(df$log_Pvae)
df$log_Pgen = log(df$Pgen)
df$log_Ppost = log(df$Ppost)

df = df %>% group_by(n_subjects, split) %>% 
    mutate(normed_Pgen = Pgen/sum(Pgen, na.rm = TRUE)) %>% 
    mutate(normed_Ppost = Ppost/sum(Ppost, na.rm = TRUE)) %>%
    mutate(normed_Pvae = Pvae/sum(Pvae, na.rm = TRUE))

df$log_normed_Pgen = log(df$normed_Pgen)
df$log_normed_Ppost = log(df$normed_Ppost)
df$log_normed_Pvae = log(df$normed_Pvae)

In [ ]:
selected_n_subjects = c(16, 64, 256, 666)
id_vars = c('n_subjects', 'split', 'log_frequency')
measure_vars = c('log_normed_Ppost', 'log_normed_Pvae')
molten = melt(df[df$n_subjects %in% selected_n_subjects,], id_vars, measure_vars, 
              variable.name='probability_name', value.name='log_probability')
molten = molten[sample(nrow(molten)),]
molten$split = factor(molten$split, levels=c('train', 'test'))
molten$n_subjects = factor(
    paste(molten$n_subjects, "subjects"),
    levels=paste(selected_n_subjects, "subjects"))

p = ggplot(molten, aes(log_frequency, log_probability, color=probability_name)) + 
    facet_grid(rows = vars(split), cols = vars(n_subjects), scales='free') + 
    geom_point(alpha=0.015) + 
    theme(aspect.ratio = 1, legend.position='bottom') + 
    coord_cartesian(ylim=c(-25,0)) + 
    guides(colour = guide_legend(override.aes = list(alpha = 1))) +
    scale_color_manual(name="probability estimator",
                         breaks=c('log_normed_Ppost', 'log_normed_Pvae'),
                         labels=c("OLGA.Q", "VAE"),
                         values=c(log_normed_Ppost = '#0000ff', log_normed_Pvae = '#ff0000')) +
                         labs(x = "log frequency", y = "log normalized probability")
p
ggsave(paste0('output/',paste(measure_vars, collapse = '_vs_'),'.png'), width=6, height=4)

In [ ]:
likes = df %>% group_by(n_subjects, split) %>% 
    summarise(
        like_Pgen = dmultinom(count, prob=normed_Pgen, log=TRUE),
        like_Ppost = dmultinom(count, prob=normed_Ppost, log=TRUE),
        like_Pvae = dmultinom(count, prob=normed_Pvae, log=TRUE),
    )
likes

In [ ]:
molten_likes = melt(likes, c('n_subjects', 'split'), value.name = 'likelihood')
molten_likes$group = paste(molten_likes$variable, molten_likes$split)

ggplot(molten_likes, aes(n_subjects, likelihood, color=variable, group=group, linetype=split)) + 
    geom_line() +
    scale_x_log10() +
    theme(aspect.ratio = phi) # golden ratio landscape

ggsave('output/multinomial_likelihoods.png', width=5, height=5*phi)

In [ ]:
# This is a little tedious: we apparently need to have a single grouping variable to feed into lmList, 
# then we need to unpack it later.

df$group = as.factor(paste(df$split, df$n_subjects))
s_Pvae = summary(lmList(log_Pvae ~ log_frequency | group, data=df[,c('log_Pvae', 'log_frequency', 'group')]))
s_Ppost = summary(lmList(log_Ppost ~ log_frequency | group, data=df[,c('log_Ppost', 'log_frequency', 'group')]))
s_Pgen = summary(lmList(log_Pgen ~ log_frequency | group, data=df[,c('log_Pgen', 'log_frequency', 'group')]))
R2 = data.frame(
    row.names = rownames(s_Pvae$df), 
    R2_Pgen = s_Pgen$r.squared,
    R2_Ppost = s_Ppost$r.squared,
    R2_Pvae = s_Pvae$r.squared)

R2$names = rownames(R2)
R2 = separate(R2, names, c('split', 'n_subjects'))
R2$n_subjects = as.integer(R2$n_subjects)
rownames(R2) = NULL
R2[order(R2$n_subjects),]

In [ ]:
molten_R2 = melt(R2, c('n_subjects', 'split'), value.name = 'R2')
molten_R2$group = paste(molten_R2$variable, molten_R2$split)
molten_R2 = molten_R2[molten_R2$variable %in% c('R2_Ppost', 'R2_Pvae'),]

ggplot(molten_R2, aes(n_subjects, R2, color=variable, group=group, linetype=split)) + 
    geom_line() +
    scale_x_log10() +
    theme(aspect.ratio = phi) +
    xlab("number of subjects") + 
    scale_color_manual(name="estimator",
                         breaks=c('R2_Pvae','R2_Ppost'),
                         labels=c("VAE","OLGA.Q"),
                         values=c(R2_Ppost = '#8da0cb', R2_Pvae = '#fc8d62')) +
    ylab(TeX("R^2"))

ggsave('output/R2.svg', width=5, height=5*phi)